<span style="font-size:1.1em;">Colab'a Google drive'ı entegre ediyoruz. Kullanılacak olan veriseti Google Drive'da bulunmaktadır</span>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<span style="font-size:1.1em;">Google Drive'ımızın root pathi</span> ```gdrive/My Drive``` <span style="font-size:1.1em;">oluyor. Proje için gerekli verisetini **mbti** adında bir klasör oluşturup içerisine yüklüyoruz. İlgili verisetinin pathi</span> ```gdrive/My Drive/mbti/all_users.csv``` <span style="font-size:1.1em;">oluyor.</span>

In [0]:
PREPROCESSED_DATASET_WITH_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_with_stemming.csv"
PREPROCESSED_DATASET_WITHOUT_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_no_stemming.csv"
PREPROCESSED_DATASET_ZEMBEREK = "gdrive/My Drive/mbti/preprocessed_dataset_zemberek"
TRIMMED_DATASET = "gdrive/My Drive/mbti/trimmed_dataset.csv"
RAW_DATASET = "gdrive/My Drive/mbti/all_users_v2.csv"

<span style="font-size:1.1em;">Hangi veriseti kullanılarak işlem yapılacaksa yukardaki pathlerden biri seçilir ve parametre olarak verilir.</span>

In [0]:
import pandas as pd 
df = pd.read_csv(PREPROCESSED_DATASET_WITH_STEMMER, sep = ';', header = 0)

In [6]:
df

,Unnamed: 0,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,0,19991991,ekşi itiraf dö dolaş gel kendi çat problem ger...,ENTJ,analysts,E,N,T,J
1,1,19991991,selda bağca ses dinleyebilmek büyük mutluluk s...,ENTJ,analysts,E,N,T,J
2,2,19991991,eski sevgili mutlu olma isteye in hiçbir ayak ...,ENTJ,analysts,E,N,T,J
3,3,19991991,veda eder a not bırakmak farklı acı mutluluk...,ENTJ,analysts,E,N,T,J
4,4,19991991,ingil aksa ara bayıldık konuşa insa sevimli ge...,ENTJ,analysts,E,N,T,J
...,...,...,...,...,...,...,...,...,...
524835,643907,zaimoglu,zla ibrahimoviç türki katil,ESFJ,sentinels,E,S,F,J
524836,643908,zaimoglu,tarih büyük yalancı şik operasyo aziz yıldır a...,ESFJ,sentinels,E,S,F,J
524837,643909,zaimoglu,akp chp koalisyo hayali koalisyo,ESFJ,sentinels,E,S,F,J
524838,643910,zaimoglu,trabzonlu insa hamsi diye fenerbahçe taraftar ...,ESFJ,sentinels,E,S,F,J


<span style="font-size:1.1em;">TF-IDF özellik vektörünün çıkartılmasında kullanılacak değişken aşağıda belirlenmiş olan parametrelerle oluşturulur.</span>

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True, norm='l2', encoding='utf-8', ngram_range=(1, 2), max_features=5000, analyzer = 'word', token_pattern=r'\w{1,}')

**Feature Extraction** 

Burada tf-idf vektörlerinin çıkartılmasında kullanılacak olan vocabulary oluşturulur.

In [0]:
import numpy as np

df['entry'] = df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

In [10]:
tfidf_vectorizer.fit(df['entry'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=True, vocabulary=None)

Veriseti train ve test olmak üzere ikiye ayrılır. Test %20 ve train %80'ini oluşturacak şekilde tüm veriseti bölünür. random_state parametresi ile tekrardan bölündüğünde bir öncekiyle aynı train ve test veri setlerinin oluşturulması sağlanır.

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['entry'], df['typeClass'], random_state = 42, test_size = 0.20)

Train ve test datasetlerinden tf-idf vektörleri çıkartılır


In [0]:
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM kullanılarak tahmin edilir.

In [0]:
from sklearn import svm

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [0]:
y_predicted

<span style="font-size:1.1em">Yapılacak tahminlerle ilgili istatistiksel verileri tutmak için</span> ```predictions_result```<span style="font-size:1.1em"> adında bir değişken oluşturulur.</span>

<span style="font-size:1.1em">Bu değişkenin yapısı aşağıdaki gibidir.</span>

```json
{
    "predicted": {
        "analysts":  { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "diplomats": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "explorers": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "sentinels": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
    }
}
```

* <span style="font-size:1.1em;">Yapılan tahminlerle ilgili verilere ulaşabilmek için</span>

    ```predictions_results['predicted']```


* <span style="font-size:1.1em;">Yapılan tahminin analyst ise:</span>

    ```predictions_results['predicted']['analysts']``` 


* <span style="font-size:1.1em;">Yapılan analyst tahmininin gerçek değerlerine erişmek için:</span>     

    ```predictions_results['predicted']['analysts']['actual']```  


* <span style="font-size:1.1em;">Test verisi, model tarafından analysts olarak tahmin edilmiştir ve bu verinin gerçek değeri de analysts'tir.</span>

    ```predictions_results['predicted']['analysts']['actual']['analysts']``` 

In [0]:
prediction_results = {'predicted': {}}  ## prediction_result['analysts'] means prediction is 'analysts'

prediction_results['predicted']['analysts']  = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['diplomats'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['explorers'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['sentinels'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}

## prediction_result['analysts']['diplomats'] means prediction is analysts but actual value is diplomats

```prediction_results```<span style="font-size:1.1em"> içerisinde tutulan sayaçların değerleri arttırılır.</span>

In [0]:
for i in range(len(y_predicted)):
  predicted_value = y_predicted[i]
  actual_value = y_actual[i]
  prediction_results['predicted'][predicted_value]['actual'][actual_value] += 1

<span style="font-size:1.1em">JSON formatına çevrilir </span>```dict``` <span style="font-size:1.1em">tipi. Bu sayede daha okunaklı bir şekilde print edilmiş olur. </span>

In [0]:
import json

print(json.dumps(prediction_results, indent = 2))

<span style="font-size:1.1em;">İlgili field extract edilir</span> ```dict``` <span style="font-size:1.1em;">yapısından.</span>

In [0]:
results = prediction_results['predicted']

<span style="font-size:1.1em;">Başarı oranı hesaplanır</span>

In [0]:
accuracy = (results['analysts']['actual']['analysts'] + results['diplomats']['actual']['diplomats'] + results['explorers']['actual']['explorers'] + results['sentinels']['actual']['sentinels']) / len(predictions)
accuracy

<span style="font-size:1.1em;">**E/I** boyutu tahmin edilir</span>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['entry'], df['I/E'], random_state = 42)  ## Geri kalanlar S, T, J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [0]:
y_predicted

array(['E', 'E', 'E', ..., 'I', 'I', 'I'], dtype='<U1')

In [0]:
predicted = {}
predicted['I'] = {'actual': {'I': 0, 'E': 0}}
predicted['E'] = {'actual': {'I': 0, 'E': 0}}
predicted

{'E': {'actual': {'E': 0, 'I': 0}}, 'I': {'actual': {'E': 0, 'I': 0}}}

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1


In [0]:
predicted

{'E': {'actual': {'E': 32427, 'I': 21303}},
 'I': {'actual': {'E': 48381, 'I': 58867}}}

In [0]:
accuracy = (predicted['E']['actual']['E'] + predicted['I']['actual']['I']) / len(predictions)

In [0]:
accuracy

0.5671209730522183

<span style="font-size:1.1em">**S/N** boyutu tahmin edilir.</span>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['entry'], df['S/N'], random_state = 42)  ## Geri kalan boyutlar: T, J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [0]:
predicted['N'] = {'actual': {'N': 0, 'S': 0}}
predicted['S'] = {'actual': {'N': 0, 'S': 0}}

predicted

{'E': {'actual': {'E': 32427, 'I': 21303}},
 'I': {'actual': {'E': 48381, 'I': 58867}},
 'N': {'actual': {'N': 0, 'S': 0}},
 'S': {'actual': {'N': 0, 'S': 0}}}

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1

In [0]:
predicted

{'E': {'actual': {'E': 32427, 'I': 21303}},
 'I': {'actual': {'E': 48381, 'I': 58867}},
 'N': {'actual': {'N': 115527, 'S': 45391}},
 'S': {'actual': {'N': 25, 'S': 35}}}

In [0]:
accuracy = (predicted['N']['actual']['N'] + predicted['S']['actual']['S']) / len(predictions)

In [0]:
accuracy

0.7178744921666315

<span style="font-size:1.1em">**T/F** boyutu tahmin edilir.</span>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['entry'], df['T/F'], random_state = 42)  ##  J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [0]:
predicted['T'] = {'actual': {'T': 0, 'F': 0}}
predicted['F'] = {'actual': {'T': 0, 'F': 0}}

predicted

{'E': {'actual': {'E': 32427, 'I': 21303}},
 'F': {'actual': {'F': 0, 'T': 0}},
 'I': {'actual': {'E': 48381, 'I': 58867}},
 'N': {'actual': {'N': 115527, 'S': 45391}},
 'S': {'actual': {'N': 25, 'S': 35}},
 'T': {'actual': {'F': 0, 'T': 0}}}

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1

In [0]:
predicted

{'E': {'actual': {'E': 32427, 'I': 21303}},
 'F': {'actual': {'F': 23204, 'T': 14732}},
 'I': {'actual': {'E': 48381, 'I': 58867}},
 'N': {'actual': {'N': 115527, 'S': 45391}},
 'S': {'actual': {'N': 25, 'S': 35}},
 'T': {'actual': {'F': 51749, 'T': 71293}}}

In [0]:
accuracy = (predicted['F']['actual']['F'] + predicted['T']['actual']['T']) / len(predictions)

In [0]:
accuracy

0.5870181018524271

<span style="font-size:1.1em">**J/P** boyutu tahmin edilir.</span>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['entry'], df['J/P'], random_state = 42) 

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [0]:
predicted['J'] = {'actual': {'J': 0, 'P': 0}}
predicted['P'] = {'actual': {'J': 0, 'P': 0}}

predicted

{'E': {'actual': {'E': 32427, 'I': 21303}},
 'F': {'actual': {'F': 23204, 'T': 14732}},
 'I': {'actual': {'E': 48381, 'I': 58867}},
 'J': {'actual': {'J': 0, 'P': 0}},
 'N': {'actual': {'N': 115527, 'S': 45391}},
 'P': {'actual': {'J': 0, 'P': 0}},
 'S': {'actual': {'N': 25, 'S': 35}},
 'T': {'actual': {'F': 51749, 'T': 71293}}}

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1

In [0]:
predicted

{'E': {'actual': {'E': 32427, 'I': 21303}},
 'F': {'actual': {'F': 23204, 'T': 14732}},
 'I': {'actual': {'E': 48381, 'I': 58867}},
 'J': {'actual': {'J': 37293, 'P': 29220}},
 'N': {'actual': {'N': 115527, 'S': 45391}},
 'P': {'actual': {'J': 40994, 'P': 53471}},
 'S': {'actual': {'N': 25, 'S': 35}},
 'T': {'actual': {'F': 51749, 'T': 71293}}}

In [0]:
accuracy = (predicted['P']['actual']['P'] + predicted['J']['actual']['J']) / len(predictions)

In [0]:
accuracy

0.5638285976965797

In [0]:
prediction_results['predicted'].update(predicted) 

prediction_results

{'predicted': {'E': {'actual': {'E': 32427, 'I': 21303}},
  'F': {'actual': {'F': 23204, 'T': 14732}},
  'I': {'actual': {'E': 48381, 'I': 58867}},
  'J': {'actual': {'J': 37293, 'P': 29220}},
  'N': {'actual': {'N': 115527, 'S': 45391}},
  'P': {'actual': {'J': 40994, 'P': 53471}},
  'S': {'actual': {'N': 25, 'S': 35}},
  'T': {'actual': {'F': 51749, 'T': 71293}},
  'analysts': {'actual': {'analysts': 38435,
    'diplomats': 28056,
    'explorers': 10084,
    'sentinels': 17542}},
  'diplomats': {'actual': {'analysts': 9521,
    'diplomats': 16309,
    'explorers': 3348,
    'sentinels': 5030}},
  'explorers': {'actual': {'analysts': 0,
    'diplomats': 1,
    'explorers': 0,
    'sentinels': 2}},
  'sentinels': {'actual': {'analysts': 95,
    'diplomats': 123,
    'explorers': 47,
    'sentinels': 190}}}}